In [1]:
import config
import tensorflow as tf
tf.app.flags.DEFINE_string('f', '', 'kernel')
from collections import deque
import model
from dataUtils import *
from logger import MyLogger
import sys
import PTB_data_reader
import time
import numpy as np
import lstm_char_cnn
import pickle
import dataloader
tf.logging.set_verbosity(tf.logging.ERROR)


logger = MyLogger("ERDMain")

W0830 20:05:18.601061 140117901190976 deprecation_wrapper.py:119] From /home/hadoop/ERD/model.py:6: The name tf.losses.Reduction is deprecated. Please use tf.compat.v1.losses.Reduction instead.

Using TensorFlow backend.


In [2]:
# load twitter data
# load_data(FLAGS.data_file_path)
load_data_fast()

#load PTB data
# word_vocab, char_vocab, word_tensors, char_tensors, max_word_length = \
#     PTB_data_reader.load_data(FLAGS.data_dir, FLAGS.max_word_length, char_vocab, eos=FLAGS.EOS)
word_vocab, char_vocab, word_tensors, char_tensors = \
    PTB_data_reader.load_data_fast()
max_word_length = FLAGS.max_word_length
train_reader = PTB_data_reader.DataReader(word_tensors['train'], char_tensors['train'],
                          FLAGS.batch_size, FLAGS.max_sent_len) 

#load sentiment analysis data
sentiReader = dataloader.SentiDataLoader(
                                        dirpath = '/home/hadoop/trainingandtestdata',
                                        trainfile = 'training.1600000.processed.noemoticon.csv', 
                                        testfile = 'testdata.manual.2009.06.14.csv', 
                                        charVocab = char_vocab
                        )
# sentiReader.load_data()
sentiReader.load_data_fast(
                        '/home/hadoop/ERD/data/senti_train_data.pickle',
                        '/home/hadoop/ERD/data/senti_train_label.pickle',
                        '/home/hadoop/ERD/data/senti_test_data.pickle',
                        '/home/hadoop/ERD/data/senti_test_label.pickle'
                          )


# (self, input_dim, hidden_dim, max_seq_len, max_word_num, class_num, action_num):
print(  FLAGS.embedding_dim, FLAGS.hidden_dim, 
            FLAGS.max_seq_len, FLAGS.max_sent_len, 
                FLAGS.class_num, FLAGS.action_num   )
logger.info(    (FLAGS.embedding_dim, FLAGS.hidden_dim, 
                    FLAGS.max_seq_len, FLAGS.max_sent_len, 
                        FLAGS.class_num, FLAGS.action_num)  )

print(get_curtime() + " Data loaded.")
logger.info(get_curtime() + " Data loaded.")

max_sent: 31 ,  max_seq_len: 101
5802 data loaded


I0830 20:05:33.144768 140117901190976 logger.py:24] (300, 200, 101, 31, 2, 2)
I0830 20:05:33.145292 140117901190976 logger.py:24] 2019-08-30 20:05:33 Data loaded.


300 200 101 31 2 2
2019-08-30 20:05:33 Data loaded.


In [3]:
# # save the Twitter data
# data = get_data()
# with open('data/data_dict.txt', 'wb') as handle:
#     pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

# # save the PTB data
# with open('data/char_tensors.txt', 'wb') as handle:
#     pickle.dump(char_tensors, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('data/word_tensors.txt', 'wb') as handle:
#     pickle.dump(word_tensors, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open('data/char_vocab.txt', 'wb') as handle:
#     pickle.dump(char_vocab, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('data/word_vocab.txt', 'wb') as handle:
#     pickle.dump(word_vocab, handle, protocol=pickle.HIGHEST_PROTOCOL)

# save the senti data
# with open('data/senti_train_data.pickle', 'wb') as handle:
#     pickle.dump(sentiReader.train_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('data/senti_train_label.pickle', 'wb') as handle:
#     pickle.dump(sentiReader.train_label, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open('data/senti_test_data.pickle', 'wb') as handle:
#     pickle.dump(sentiReader.test_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('data/senti_test_label.pickle', 'wb') as handle:
#     pickle.dump(sentiReader.test_label, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [3]:
import importlib
importlib.reload(lstm_char_cnn)
importlib.reload(model)

<module 'model' from '/home/hadoop/ERD/model.py'>

In [4]:
w2v = lstm_char_cnn.WordEmbedding(
                max_word_length = FLAGS.max_char_num , 
                char_vocab_size = char_vocab.size, 
                char_embed_size = FLAGS.char_embed_size, 
                kernels = eval(FLAGS.kernels), 
                kernel_features = eval(FLAGS.kernel_features), 
                num_highway_layers = FLAGS.highway_layers,
                embedding_dim = FLAGS.embedding_dim
            )
lstm_lm = lstm_char_cnn.LSTM_LM(
            batch_size = FLAGS.batch_size, 
            num_unroll_steps = FLAGS.max_sent_len, 
            rnn_size = FLAGS.embedding_dim, 
            num_rnn_layers = FLAGS.rnn_layers, 
            word_vocab_size = word_vocab.size
        )

char_train_graph = lstm_char_cnn.infer_train_model(
                    w2v, lstm_lm, 
                    batch_size = FLAGS.batch_size, 
                    num_unroll_steps = FLAGS.max_sent_len, 
                    max_word_length = FLAGS.max_char_num, 
                    learning_rate = FLAGS.learning_rate,
                    max_grad_norm = FLAGS.max_grad_norm
                 )
#sentiment analysis model
s_model = model.SentiModel(FLAGS.hidden_dim, 5)
senti_train_graph = model.InferSentiTrainGraph(
                        w2v, 
                        lstm_lm, 
                        s_model, 
                        max_word_num = FLAGS.max_sent_len, 
                        max_char_num = FLAGS.max_char_num, 
                        hidden_dim = FLAGS.hidden_dim, 
                        sent_num = FLAGS.sent_num,
                        embedding_dim = FLAGS.embedding_dim
                    )

input_: Tensor("input:0", shape=(20, 31, 21), dtype=int32)
input_cnn: Tensor("Embedding_1/CNN_OUT/add_7:0", shape=(620, 1100), dtype=float32)
input_: Tensor("Placeholder_1:0", shape=(?, 31, 21), dtype=int32)
input_cnn: Tensor("Embedding_2/CNN_OUT/add_7:0", shape=(?, 1100), dtype=float32)


In [19]:
importlib.reload(lstm_char_cnn)
importlib.reload(model)

<module 'model' from '/home/hadoop/ERD/model.py'>

In [5]:

# df model
rdm_model = model.RDM_Model(
                max_seq_len = FLAGS.max_seq_len, 
                max_word_num = FLAGS.max_sent_len, 
                embedding_dim = FLAGS.embedding_dim, 
                hidden_dim = FLAGS.hidden_dim
            )
rdm_train_graph = model.InferRDMTrainGraph(
                        w2v, lstm_lm, s_model, rdm_model, 
                        batchsize=FLAGS.batch_size,
                        max_seq_len = FLAGS.max_seq_len, 
                        max_word_num = FLAGS.max_sent_len, 
                        max_char_num = FLAGS.max_char_num, 
                        hidden_dim = FLAGS.hidden_dim, 
                        embedding_dim = FLAGS.embedding_dim,
                        class_num = FLAGS.class_num
                )

# rl model
cm_model = model.CM_Model(
                    max_word_num = FLAGS.max_sent_len, 
                    embedding_dim = FLAGS.embedding_dim, 
                    hidden_dim = FLAGS.hidden_dim, 
                    action_num = FLAGS.action_num
            )
cm_train_graph = model.InferCMTrainGraph(
                        w2v, s_model, rdm_model, cm_model, 
                        max_word_num = FLAGS.max_sent_len, 
                        embedding_dim = FLAGS.embedding_dim, 
                        hidden_dim = FLAGS.hidden_dim, 
                        action_num = FLAGS.action_num
                    )

saver = tf.train.Saver(tf.global_variables(), max_to_keep=4)
sess = tf.Session()
with sess.as_default():
    sess.run(tf.global_variables_initializer())

summary_writer = tf.summary.FileWriter(FLAGS.train_dir, graph=sess.graph)

x_reshape: Tensor("Reshape_2:0", shape=(2020, 31, 21), dtype=int32)
input_: Tensor("Reshape_2:0", shape=(2020, 31, 21), dtype=int32)
input_cnn: Tensor("Embedding_3/CNN_OUT/add_7:0", shape=(62620, 1100), dtype=float32)
x_embedding: Tensor("Embedding_3/CNN_OUT/add_7:0", shape=(62620, 1100), dtype=float32)
cnn_outs: Tensor("Reshape_3:0", shape=(2020, 31, 1100), dtype=float32)
RDM words_embedding: Tensor("transpose_1:0", shape=(2020, 31, 300), dtype=float32)
x_senti: Tensor("SentiModel_1/Max:0", shape=(2020, 200), dtype=float32)
pooled_rl_input: Tensor("Reshape_5:0", shape=(?, 200), dtype=float32)
rl_state: Tensor("rl_states:0", shape=(?, 200), dtype=float32)


In [7]:
import importlib
importlib.reload(lstm_char_cnn)

<module 'lstm_char_cnn' from '/home/hadoop/ERD/lstm_char_cnn.py'>

In [ ]:
lstm_char_cnn.Train_Char_Model(sess, char_train_graph, train_reader, saver, summary_writer)

     5: 0 [    5/ 1499], train_loss/perplexity = 9.15729713/9483.3896484 secs/batch = 0.5930s, grad.norm=0.12490036
    10: 0 [   10/ 1499], train_loss/perplexity = 9.08498096/8821.7978516 secs/batch = 0.6202s, grad.norm=0.13333045
    15: 0 [   15/ 1499], train_loss/perplexity = 8.98046017/7946.2880859 secs/batch = 0.7101s, grad.norm=0.18570276
    20: 0 [   20/ 1499], train_loss/perplexity = 8.63869190/5645.9394531 secs/batch = 0.5504s, grad.norm=0.43776885
    25: 0 [   25/ 1499], train_loss/perplexity = 7.53612280/1874.5479736 secs/batch = 0.6260s, grad.norm=0.47992179
    30: 0 [   30/ 1499], train_loss/perplexity = 7.29143143/1467.6700439 secs/batch = 0.5361s, grad.norm=1.12251890
    35: 0 [   35/ 1499], train_loss/perplexity = 7.44433117/1710.1411133 secs/batch = 0.5397s, grad.norm=1.29667437
    40: 0 [   40/ 1499], train_loss/perplexity = 7.20252466/1342.8166504 secs/batch = 0.5403s, grad.norm=1.38146412
    45: 0 [   45/ 1499], train_loss/perplexity = 7.23605251/1388.6016846

   365: 0 [  365/ 1499], train_loss/perplexity = 6.63771820/763.3511963 secs/batch = 0.5380s, grad.norm=1.15613115
   370: 0 [  370/ 1499], train_loss/perplexity = 6.57711983/718.4670410 secs/batch = 0.5411s, grad.norm=0.44019940
   375: 0 [  375/ 1499], train_loss/perplexity = 6.43539858/623.5310669 secs/batch = 0.5414s, grad.norm=0.33791822
   380: 0 [  380/ 1499], train_loss/perplexity = 6.64196396/766.5990601 secs/batch = 0.5434s, grad.norm=0.52224058
   385: 0 [  385/ 1499], train_loss/perplexity = 6.61221409/744.1287842 secs/batch = 0.5400s, grad.norm=0.50121397
   390: 0 [  390/ 1499], train_loss/perplexity = 6.81604147/912.3662109 secs/batch = 0.5370s, grad.norm=0.35354772
   395: 0 [  395/ 1499], train_loss/perplexity = 6.80333662/900.8480835 secs/batch = 0.5420s, grad.norm=0.41359380
   400: 0 [  400/ 1499], train_loss/perplexity = 6.71813107/827.2699585 secs/batch = 0.5535s, grad.norm=0.49285099
   405: 0 [  405/ 1499], train_loss/perplexity = 6.58374310/723.2414551 secs/bat

   725: 0 [  725/ 1499], train_loss/perplexity = 6.52330303/680.8234863 secs/batch = 0.5357s, grad.norm=0.67174274
   730: 0 [  730/ 1499], train_loss/perplexity = 6.60094213/735.7880859 secs/batch = 0.5373s, grad.norm=0.45806772
   735: 0 [  735/ 1499], train_loss/perplexity = 6.57344103/715.8287964 secs/batch = 0.5337s, grad.norm=0.41944647
   740: 0 [  740/ 1499], train_loss/perplexity = 6.57568359/717.4359131 secs/batch = 0.5410s, grad.norm=0.30755678
   745: 0 [  745/ 1499], train_loss/perplexity = 6.48124552/652.7835083 secs/batch = 0.5372s, grad.norm=0.41048518
   750: 0 [  750/ 1499], train_loss/perplexity = 6.62926912/756.9287720 secs/batch = 0.5467s, grad.norm=0.34821105
   755: 0 [  755/ 1499], train_loss/perplexity = 6.50807333/670.5332642 secs/batch = 0.5429s, grad.norm=0.32785997
   760: 0 [  760/ 1499], train_loss/perplexity = 6.28847456/538.3314819 secs/batch = 0.5420s, grad.norm=0.33857116
   765: 0 [  765/ 1499], train_loss/perplexity = 6.53475618/688.6658325 secs/bat

  1085: 0 [ 1085/ 1499], train_loss/perplexity = 6.68645906/801.4792480 secs/batch = 0.5370s, grad.norm=0.32147256
  1090: 0 [ 1090/ 1499], train_loss/perplexity = 6.43140650/621.0468140 secs/batch = 0.5377s, grad.norm=0.34510538
  1095: 0 [ 1095/ 1499], train_loss/perplexity = 6.62986803/757.3822021 secs/batch = 0.5432s, grad.norm=0.32318753
  1100: 0 [ 1100/ 1499], train_loss/perplexity = 6.62116766/750.8212891 secs/batch = 0.5441s, grad.norm=0.29048184
  1105: 0 [ 1105/ 1499], train_loss/perplexity = 6.53903818/691.6210327 secs/batch = 0.5483s, grad.norm=0.34881800
  1110: 0 [ 1110/ 1499], train_loss/perplexity = 6.60372019/737.8349609 secs/batch = 0.5392s, grad.norm=0.28734133
  1115: 0 [ 1115/ 1499], train_loss/perplexity = 6.36170435/579.2327271 secs/batch = 0.5519s, grad.norm=0.32726750
  1120: 0 [ 1120/ 1499], train_loss/perplexity = 6.60577869/739.3553467 secs/batch = 0.5395s, grad.norm=0.40353474
  1125: 0 [ 1125/ 1499], train_loss/perplexity = 6.54308033/694.4223633 secs/bat

  1445: 0 [ 1445/ 1499], train_loss/perplexity = 6.31693411/553.8722534 secs/batch = 0.5748s, grad.norm=0.35198790
  1450: 0 [ 1450/ 1499], train_loss/perplexity = 6.31810999/554.5239258 secs/batch = 0.5382s, grad.norm=0.29216608
  1455: 0 [ 1455/ 1499], train_loss/perplexity = 6.54954863/698.9286499 secs/batch = 0.5433s, grad.norm=0.37018383
  1460: 0 [ 1460/ 1499], train_loss/perplexity = 6.58060026/720.9719849 secs/batch = 0.5425s, grad.norm=0.38409078
  1465: 0 [ 1465/ 1499], train_loss/perplexity = 6.81263685/909.2652588 secs/batch = 0.5421s, grad.norm=0.37625739
  1470: 0 [ 1470/ 1499], train_loss/perplexity = 6.41194677/609.0782471 secs/batch = 0.5395s, grad.norm=0.46633723
  1475: 0 [ 1475/ 1499], train_loss/perplexity = 6.37993383/589.8886719 secs/batch = 0.5380s, grad.norm=0.33547100
  1480: 0 [ 1480/ 1499], train_loss/perplexity = 6.52528381/682.1733398 secs/batch = 0.5355s, grad.norm=0.33582419
  1485: 0 [ 1485/ 1499], train_loss/perplexity = 6.23986483/512.7891846 secs/bat

...

 13671: 9 [  180/ 1499], train_loss/perplexity = 5.04420137/155.1203613 secs/batch = 0.5457s, grad.norm=0.64704633
 13676: 9 [  185/ 1499], train_loss/perplexity = 4.87427473/130.8791962 secs/batch = 0.5538s, grad.norm=0.64361888
 13681: 9 [  190/ 1499], train_loss/perplexity = 4.96886778/143.8639069 secs/batch = 0.6234s, grad.norm=0.63555056
 13686: 9 [  195/ 1499], train_loss/perplexity = 5.12590551/168.3264923 secs/batch = 0.5865s, grad.norm=0.64932841
 13691: 9 [  200/ 1499], train_loss/perplexity = 5.06811476/158.8745270 secs/batch = 0.5765s, grad.norm=0.65143234
 13696: 9 [  205/ 1499], train_loss/perplexity = 4.95790339/142.2951508 secs/batch = 0.5474s, grad.norm=0.63576275
 13701: 9 [  210/ 1499], train_loss/perplexity = 4.80340528/121.9249039 secs/batch = 0.5651s, grad.norm=0.61377412
 13706: 9 [  215/ 1499], train_loss/perplexity = 5.02237654/151.7715607 secs/batch = 0.5488s, grad.norm=0.62258363
 13711: 9 [  220/ 1499], train_loss/perplexity = 4.74233580/114.7018127 secs/bat

 14031: 9 [  540/ 1499], train_loss/perplexity = 4.76116371/116.8818665 secs/batch = 0.6153s, grad.norm=0.64626020
 14036: 9 [  545/ 1499], train_loss/perplexity = 4.60927677/100.4115067 secs/batch = 0.6308s, grad.norm=0.63292873
 14041: 9 [  550/ 1499], train_loss/perplexity = 5.06042814/157.6580048 secs/batch = 0.6459s, grad.norm=0.67764020
 14046: 9 [  555/ 1499], train_loss/perplexity = 4.83798122/126.2142944 secs/batch = 0.6338s, grad.norm=0.64376473
 14051: 9 [  560/ 1499], train_loss/perplexity = 5.14847517/172.1687622 secs/batch = 0.6210s, grad.norm=0.61732525
 14056: 9 [  565/ 1499], train_loss/perplexity = 5.14686537/171.8918304 secs/batch = 0.7058s, grad.norm=0.65208060
 14061: 9 [  570/ 1499], train_loss/perplexity = 5.05190611/156.3201447 secs/batch = 0.6231s, grad.norm=0.60726982
 14066: 9 [  575/ 1499], train_loss/perplexity = 5.12521744/168.2107086 secs/batch = 0.6479s, grad.norm=0.60213804
 14071: 9 [  580/ 1499], train_loss/perplexity = 4.79536629/120.9486771 secs/bat

 14391: 9 [  900/ 1499], train_loss/perplexity = 4.64513111/104.0770111 secs/batch = 0.6603s, grad.norm=0.57509750
 14396: 9 [  905/ 1499], train_loss/perplexity = 4.80580807/122.2182083 secs/batch = 0.5976s, grad.norm=0.60679024
 14401: 9 [  910/ 1499], train_loss/perplexity = 4.97587109/144.8749695 secs/batch = 0.6771s, grad.norm=0.65896565
 14406: 9 [  915/ 1499], train_loss/perplexity = 4.84910440/127.6260376 secs/batch = 0.6919s, grad.norm=0.70013601
 14411: 9 [  920/ 1499], train_loss/perplexity = 4.54213476/93.8910217 secs/batch = 0.6294s, grad.norm=0.58493280
 14416: 9 [  925/ 1499], train_loss/perplexity = 4.76387405/117.1990814 secs/batch = 0.6143s, grad.norm=0.61755979
 14421: 9 [  930/ 1499], train_loss/perplexity = 4.84400415/126.9767685 secs/batch = 1.0978s, grad.norm=0.65597075
 14426: 9 [  935/ 1499], train_loss/perplexity = 4.92137527/137.1911621 secs/batch = 0.6020s, grad.norm=0.63614130
 14431: 9 [  940/ 1499], train_loss/perplexity = 4.53020668/92.7777328 secs/batch

 14751: 9 [ 1260/ 1499], train_loss/perplexity = 4.85047007/127.8004532 secs/batch = 0.5699s, grad.norm=0.64646280
 14756: 9 [ 1265/ 1499], train_loss/perplexity = 4.88884640/132.8002930 secs/batch = 0.6450s, grad.norm=0.64731294
 14761: 9 [ 1270/ 1499], train_loss/perplexity = 4.83886576/126.3259888 secs/batch = 0.6747s, grad.norm=0.62575245
 14766: 9 [ 1275/ 1499], train_loss/perplexity = 4.82626534/124.7442093 secs/batch = 0.5872s, grad.norm=0.63883382
 14771: 9 [ 1280/ 1499], train_loss/perplexity = 4.49070454/89.1842575 secs/batch = 0.6483s, grad.norm=0.61034226
 14776: 9 [ 1285/ 1499], train_loss/perplexity = 5.03062391/153.0284576 secs/batch = 0.6164s, grad.norm=0.66634321
 14781: 9 [ 1290/ 1499], train_loss/perplexity = 4.73037291/113.3378220 secs/batch = 1.0736s, grad.norm=0.63523763
 14786: 9 [ 1295/ 1499], train_loss/perplexity = 4.77402163/118.3944244 secs/batch = 0.6140s, grad.norm=0.68821311
 14791: 9 [ 1300/ 1499], train_loss/perplexity = 4.85412788/128.2687836 secs/batc

 15110: 10 [  120/ 1499], train_loss/perplexity = 4.55523777/95.1293716 secs/batch = 0.5556s, grad.norm=0.61588126
 15115: 10 [  125/ 1499], train_loss/perplexity = 4.66785336/106.4689484 secs/batch = 0.5508s, grad.norm=0.66774142
 15120: 10 [  130/ 1499], train_loss/perplexity = 4.80306721/121.8836899 secs/batch = 0.6186s, grad.norm=0.67160994
 15125: 10 [  135/ 1499], train_loss/perplexity = 4.85398006/128.2498169 secs/batch = 0.6224s, grad.norm=0.70908475
 15130: 10 [  140/ 1499], train_loss/perplexity = 4.86500502/129.6715851 secs/batch = 0.5875s, grad.norm=0.65894848
 15135: 10 [  145/ 1499], train_loss/perplexity = 4.51426077/91.3100433 secs/batch = 0.5584s, grad.norm=0.67035639
 15140: 10 [  150/ 1499], train_loss/perplexity = 4.70496702/110.4946442 secs/batch = 0.5820s, grad.norm=0.66384786
 15145: 10 [  155/ 1499], train_loss/perplexity = 4.87296438/130.7078094 secs/batch = 0.5429s, grad.norm=0.62957329
 15150: 10 [  160/ 1499], train_loss/perplexity = 5.10725212/165.2157440 s

In [5]:
import importlib

In [6]:
importlib.reload(model)
model.TrainSentiModel(sess, saver, senti_train_graph, sentiReader, FLAGS.batch_size, FLAGS.batch_size)

data_X shape: (20, 31, 21)
data_Y shape: (20, 3)
sentX shape: Tensor("Placeholder_1:0", shape=(?, 31, 21), dtype=int32)
sent_y, shape: Tensor("Placeholder_2:0", shape=(?, 3), dtype=float32)
data_X shape: (20, 31, 21)
data_Y shape: (20, 3)
sentX shape: Tensor("Placeholder_1:0", shape=(?, 31, 21), dtype=int32)
sent_y, shape: Tensor("Placeholder_2:0", shape=(?, 3), dtype=float32)
data_X shape: (20, 31, 21)
data_Y shape: (20, 3)
sentX shape: Tensor("Placeholder_1:0", shape=(?, 31, 21), dtype=int32)
sent_y, shape: Tensor("Placeholder_2:0", shape=(?, 3), dtype=float32)
data_X shape: (20, 31, 21)
data_Y shape: (20, 3)
sentX shape: Tensor("Placeholder_1:0", shape=(?, 31, 21), dtype=int32)
sent_y, shape: Tensor("Placeholder_2:0", shape=(?, 3), dtype=float32)
data_X shape: (20, 31, 21)
data_Y shape: (20, 3)
sentX shape: Tensor("Placeholder_1:0", shape=(?, 31, 21), dtype=int32)
sent_y, shape: Tensor("Placeholder_2:0", shape=(?, 3), dtype=float32)
data_X shape: (20, 31, 21)
data_Y shape: (20, 3)
s

NameError: name 'logger' is not defined

In [6]:
importlib.reload(model)
model.TrainRDMModel(sess, rdm_train_graph, 0.7, 100)

Unknown char: ’
Word: that’s
Unknown char: ’
Word: said..he’s
Unknown char: ’
Word: they’re
Unknown char: "
Word: a"cleric
Unknown char: è
Word: ministère
Unknown char: ’
Word: l’intérieur
Unknown char: é
Word: l’intérieur
Unknown char: ’
Word: today’s
Unknown char: ’
Word: don’t
Unknown char: ’
Word: wouldn’t
Unknown char: "
Word: been"justifying
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: e░n░d
Unknown char: ░
Word: e░n░d
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: e░n░d
Unknown char: ░
Word: e░n░d
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknow

NameError: name 'logger' is not defined

In [9]:
importlib.reload(model)
model.TrainCMModel(sess, rdm_train_graph, cm_train_graph, 0.3, 100)

in RL the begining
Unknown char: ’
Word: that’s
Unknown char: ’
Word: said..he’s
Unknown char: ’
Word: they’re
Unknown char: "
Word: a"cleric
Unknown char: è
Word: ministère
Unknown char: ’
Word: l’intérieur
Unknown char: é
Word: l’intérieur
Unknown char: ’
Word: today’s
Unknown char: ’
Word: don’t
Unknown char: ’
Word: wouldn’t
Unknown char: "
Word: been"justifying
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: e░n░d
Unknown char: ░
Word: e░n░d
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: e░n░d
Unknown char: ░
Word: e░n░d
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Wor

Unknown char: ề
Word: iều
Unknown char: ò
Word: hòa
Unknown char: ầ
Word: trần
Unknown char: =
Word: massmedia=junkfood
Unknown char: é
Word: gérard
Unknown char: 🐸
Word: french🐸frog
Unknown char: é
Word: défendre
Unknown char: é
Word: démocratie
Unknown char: ä
Word: gefährlich
Unknown char: ’
Word: that’s
Unknown char: é
Word: célèbrités
Unknown char: è
Word: célèbrités
Unknown char: é
Word: célèbrités
Unknown char: è
Word: ramènent
Unknown char: ê
Word: même
Unknown char: "
Word: issues."i
Unknown char: ’
Word: today’s
Unknown char: ’
Word: today’s
Unknown char: "
Word: unemplymt."they"deserve
Unknown char: "
Word: unemplymt."they"deserve
Unknown char: ’
Word: it’s
Unknown char: ’
Word: it’s
Unknown char: "
Word: i"m
Unknown char: é
Word: protégés
Unknown char: é
Word: protégés
Unknown char: ’
Word: they’re
Unknown char: …
Word: display…as
Unknown char: …
Word: staff…publish
Unknown char: …
Word: week…1m
Unknown char: "
Word: important"...more
Unknown char: "
Word: the"religion
Unkn

Unknown char: é
Word: spéisiúl
Unknown char: ú
Word: spéisiúl
Unknown char: ó
Word: fóill
Unknown char: í
Word: shuíomh
Unknown char: é
Word: insécurit
Unknown char: ’
Word: md’s
Unknown char: é
Word: astérix
Unknown char: é
Word: résistant
Unknown char: è
Word: très
Unknown char: á
Word: están
Unknown char: ó
Word: expresión
Unknown char: é
Word: también
Unknown char: í
Word: crítica
Unknown char: ’
Word: can’t
Unknown char: ’
Word: it’s
Unknown char: "
Word: article"how
Unknown char: =
Word: it=a
Unknown char: é
Word: l'amérique
Unknown char: "
Word: you're*...."as
Unknown char: "
Word: faith"when
Unknown char: ö
Word: angehörigen
Unknown char: ü
Word: mitgefühl
Unknown char: ü
Word: für
Unknown char: ’
Word: it’s
Unknown char: é
Word: aéreos
Unknown char: ’
Word: aren’t
Unknown char: ’
Word: that’s
Unknown char: ’
Word: gov’t
Unknown char: é
Word: norvégien
Unknown char: é
Word: dégouts
Unknown char: ’
Word: it’s
Unknown char: ’
Word: you’re
Unknown char: ’
Word: allan’s
Unknown cha

Unknown char: ’
Word: wolinski’s
Unknown char: ’
Word: don’t
Unknown char: ’
Word: wolinski’s
Unknown char: ’
Word: don’t
Unknown char: ’
Word: wolinski’s
Unknown char: ’
Word: don’t
Unknown char: ’
Word: wolinski’s
Unknown char: ’
Word: don’t
Unknown char: ’
Word: wolinski’s
Unknown char: ’
Word: don’t
Unknown char: é
Word: démocratie
Unknown char: é
Word: décidément
Unknown char: é
Word: décidément
Unknown char: ―
Word: conviction―just
Unknown char: ―
Word: conviction―just
Unknown char: ’
Word: don’t
Unknown char: ë
Word: dammartin-en-goël
Unknown char: ’
Word: you’re
Unknown char: ’
Word: it’s
Unknown char: ’
Word: they’re
Unknown char: ’
Word: aren’t
Unknown char: ’
Word: it’s
Unknown char: ’
Word: it’s
Unknown char: í
Word: víctimas
Unknown char: 😷
Word: daily😷😷😷your
Unknown char: 😷
Word: daily😷😷😷your
Unknown char: 😷
Word: daily😷😷😷your
Unknown char: =
Word: violations=dismissal
Unknown char: ü
Word: türkiye’de
Unknown char: ’
Word: türkiye’de
Unknown char: ş
Word: işi̇d
Unknown ch

Unknown char: é
Word: priorités
Unknown char: é
Word: déplac
Unknown char: â
Word: mankind.âll
Unknown char: =
Word: sharialaw=hamas=isis
Unknown char: =
Word: sharialaw=hamas=isis
Unknown char: =
Word: haram=hezbollah
Unknown char: "
Word: bold"?...trying
Unknown char: =
Word: lies=lies=lies
Unknown char: =
Word: lies=lies=lies
Unknown char: ’
Word: won’t
Unknown char: ’
Word: i’m
Unknown char: ’
Word: isn’t
Unknown char: ’
Word: us’s
Unknown char: ’
Word: isn’t
Unknown char: ’
Word: us’s
Unknown char: =
Word: suspects=more
Unknown char: é
Word: téméraire
Unknown char: é
Word: téméraire
Unknown char: ’
Word: i’d
Unknown char: ’
Word: wasn’t
Unknown char: è
Word: frère
Unknown char: é
Word: montée
Unknown char: í
Word: policía
Unknown char: í
Word: polícia
Unknown char: á
Word: austrália
Unknown char: é
Word: mantém
Unknown char: á
Word: vários
Unknown char: é
Word: reféns
Unknown char: ’
Word: won’t
Unknown char: ’
Word: he’s
Unknown char: ’
Word: he’s
Unknown char: "
Word: w/"reporte

Unknown char: =
Word: americans=full
Unknown char: ä
Word: väl
Unknown char: ä
Word: självklarhet
Unknown char: ’
Word: you’re
Unknown char: ’
Word: l’histoire
Unknown char: ’
Word: envoy’s
Unknown char: ’
Word: don’t
Unknown char: ’
Word: there’s
Unknown char: ’
Word: don’t
Unknown char: ’
Word: doesn’t
Unknown char: ’
Word: i’ve
Unknown char: ’
Word: that’s
Unknown char: ’
Word: don’t
Unknown char: ’
Word: i’d
Unknown char: ’
Word: it’s
Unknown char: "
Word: indication"of
Unknown char: ’
Word: i’m
Unknown char: ’
Word: i’m
Unknown char: ’
Word: people’s
Unknown char: ’
Word: let’s
Unknown char: ’
Word: i’ve
Unknown char: ’
Word: bush’s
Unknown char: ’
Word: bush’s
Unknown char: ’
Word: there’s
Unknown char: ’
Word: someone’s
Unknown char: ’
Word: it’s
Unknown char: ’
Word: there’s
Unknown char: ’
Word: someone’s
Unknown char: ’
Word: gun.it’s
Unknown char: ü
Word: sürüyor
Unknown char: ü
Word: sürüyor
Unknown char: é
Word: dénonce
Unknown char: "
Word: much"im
Unknown char: ’
Word: y

Unknown char: =
Word: murder=intentional
Unknown char: ’
Word: canada’s
Unknown char: ä
Word: jämföra
Unknown char: ö
Word: jämföra
Unknown char: å
Word: någon
Unknown char: ö
Word: förstår
Unknown char: å
Word: förstår
Unknown char: å
Word: sånt
Unknown char: ä
Word: här
Unknown char: ö
Word: för
Unknown char: ö
Word: förlorare
Unknown char: ä
Word: väldigt
Unknown char: é
Word: l'amérique
Unknown char: è
Word: derrière
Unknown char: ’
Word: i’d
Unknown char: ’
Word: there’s
Unknown char: ’
Word: doesn’t
Unknown char: ’
Word: it’s
Unknown char: ’
Word: don’t
Unknown char: ’
Word: can’t
Unknown char: ’
Word: you’re
Unknown char: ’
Word: i’m
Unknown char: ö
Word: möglich
Unknown char: ¨
Word: it¨s
Unknown char: â
Word: islâmico
Unknown char: ã
Word: são
Unknown char: ã
Word: estão
Unknown char: ’
Word: i’d
Unknown char: "
Word: war."once
Unknown char: "
Word: you"re
Unknown char: "
Word: in"jean
Unknown char: á
Word: barbárie
Unknown char: á
Word: más
Unknown char: ñ
Word: viñetas
Unkno

Unknown char: =
Word: dreadheads=savages
Unknown char: ’
Word: don’t
Unknown char: ’
Word: that’s
Unknown char: ’
Word: it’s
Unknown char: ’
Word: wouldn’t
Unknown char: ’
Word: i’m
Unknown char: ’
Word: doesn’t
Unknown char: ’
Word: i’m
Unknown char: ’
Word: it’s
Unknown char: ’
Word: i’m
Unknown char: ’
Word: i’m
Unknown char: ’
Word: it’s
Unknown char: ’
Word: that’s
Unknown char: â
Word: châtiment
Unknown char: é
Word: c'était
Unknown char: é
Word: véritable
Unknown char: è
Word: succès
Unknown char: =
Word: hispanicsasians=democrat
Unknown char: =
Word: christians=republicans
Unknown char: =
Word: christians=dems
Unknown char: ü
Word: mitgefühl
Unknown char: ö
Word: gehört
Unknown char: ö
Word: angehörigen
Unknown char: ̇
Word: i̇srailli
Unknown char: ü
Word: tümgeneral
Unknown char: ç
Word: koçavi
Unknown char: ü
Word: türkiye’de
Unknown char: ’
Word: türkiye’de
Unknown char: 
make: gun
Unknown char: 
rewardsun
Unknown char: 
puttingead
Unknown char: ’
Word: i’m
Unknown char: ’
W

KeyboardInterrupt: 